<a href="https://colab.research.google.com/github/gekoramy/uni.deep-learning/blob/standard-finetuning/standard_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Standard finetuning
In this notebook we propose the straightforward solution to fine tune CLIP. The general idea is to add linear layer(s) on top of the 1024 visual features of CLIP.

Dependences

In [1]:
%%shell
tee requirements.txt << END
ftfy
jaxtyping
jupyter
matplotlib
pydantic
regex
torch
torchvision
torchinfo
tqdm
END

pip install -q -r requirements.txt
pip install -q git+https://github.com/openai/CLIP.git
pip install -q ultralytics

ftfy
jaxtyping
jupyter
matplotlib
pydantic
regex
torch
torchvision
torchinfo
tqdm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.0/613.0 kB 5.7 MB/s eta 0:00:00


In [2]:
import clip
import json
import os
import pickle
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import PIL
import itertools as it

from datetime import datetime
from jaxtyping import Float, UInt, Int
from pydantic.dataclasses import dataclass
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.utils import draw_bounding_boxes
from torchvision.io import read_image
from torchinfo import summary
from typing import Literal, Callable, Mapping, TypeVar
from tqdm import tqdm
from timeit import default_timer as timer
from torch.utils.tensorboard import SummaryWriter

In [3]:
device: Literal['cpu', 'cuda'] = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)
device

'cpu'

## Load the dataset
First of all we have to load the dataset.

In [4]:
%%shell
if ! [ -d dataset ]; then
  mkdir dataset &&
  gdown 1P8a1g76lDJ8cMIXjNDdboaRR5-HsVmUb &&
  tar -xf refcocog.tar.gz -C dataset &&
  rm refcocog.tar.gz
fi

Downloading...
From: https://drive.google.com/uc?id=1P8a1g76lDJ8cMIXjNDdboaRR5-HsVmUb
To: /content/refcocog.tar.gz
100% 13.5G/13.5G [05:29<00:00, 40.9MB/s]


Folder paths

In [5]:
root = os.path.join('dataset', 'refcocog', '')
data_instances = os.path.join(root, 'annotations', 'instances.json')
data_refs = os.path.join(root, 'annotations', 'refs(umd).p')
data_images = os.path.join(root, 'images', '')

Type declaration

In [6]:
I = TypeVar('I')
P = TypeVar('P')
B = TypeVar('B')
T = TypeVar('T')

Img = UInt[torch.Tensor, 'C W H']
BBox = UInt[torch.Tensor, '4']
Split = Literal['train', 'test', 'val']

@dataclass
class Info:
    description: str  # This is stable 1.0 version of the 2014 MS COCO dataset.
    url: str  # http://mscoco.org/
    version: str  # 1.0
    year: int  # 2014
    contributor: str  # Microsoft COCO group
    date_created: datetime  # 2015-01-27 09:11:52.357475

@dataclass
class Image:
    license: int  # each image has an associated licence id
    file_name: str  # file name of the image
    coco_url: str  # example http://mscoco.org/images/131074
    height: int
    width: int
    flickr_url: str  # example http://farm9.staticflickr.com/8308/7908210548_33e
    id: int  # id of the imag
    date_captured: datetime  # example '2013-11-21 01:03:06'

@dataclass
class License:
    url: str  # example http://creativecommons.org/licenses/by-nc-sa/2.0/
    id: int  # id of the licence
    name: str  # example 'Attribution-NonCommercial-ShareAlike License

@dataclass
class Annotation:
    #segmentation: list[list[float]]  # description of the mask; example [[44.17, 217.83, 36.21, 219.37, 33.64, 214.49, 31.08, 204.74, 36.47, 202.68, 44.17, 203.2]]
    area: float  # number of pixel of the described object
    iscrowd: Literal[1, 0]  # Crowd annotations (iscrowd=1) are used to label large groups of objects (e.g. a crowd of people)
    image_id: int  # id of the target image
    bbox: tuple[float, float, float, float]  # bounding box coordinates [xmin, ymin, width, height]
    category_id: int
    id: int  # annotation id

@dataclass
class Category:
    supercategory: str  # example 'vehicle'
    id: int  # category id
    name: str  # example 'airplane'

@dataclass
class Instances:
    info: Info
    images: list[Image]
    licenses: list[License]
    annotations: list[Annotation]
    categories: list[Category]

@dataclass
class Sentence:
    tokens: list[str]  # tokenized version of referring expression
    raw: str  # unprocessed referring expression
    sent: str  # referring expression with mild processing, lower case, spell correction, etc.
    sent_id: int  # unique referring expression id

@dataclass
class Ref:
    image_id: int  # unique image id
    split: Split
    sentences: list[Sentence]
    file_name: str  # file name of image relative to img_root
    category_id: int  # object category label
    ann_id: int  # id of object annotation in instance.json
    sent_ids: list[int]  # same ids as nested sentences[...][sent_id]
    ref_id: int  # unique id for refering expression

In [7]:
#@dataclass
#class Prediction:
#  image
#  description: list[str]  # natural language descriptions of the area of interest
#  ground_truth_bbox: tuple[float, float, float, float] # ground truth bounding box
#  output_bbox: tuple[float, float, float, float] # predicted bounding box

class Prediction:
  def __init__(self, image, description, ground_truth_bbox, output_bbox):
    self.image = image
    self.description = description
    self.ground_truth_bbox = ground_truth_bbox
    self.output_bbox = output_bbox

Read the dataset infos

In [8]:
def fix_ref(x: Ref) -> Ref:
    x.file_name = fix_filename(x.file_name)
    return x


def fix_filename(x: str) -> str:
    """
    :param x: COCO_..._[image_id]_[annotation_id].jpg
    :return:  COCO_..._[image_id].jpg
    """
    return re.sub('_\d+\.jpg$', '.jpg', x)

In [9]:
with open(data_refs, 'rb') as f:
    raw = pickle.load(f)

In [10]:
refs: list[Ref] = [
    fix_ref(Ref(**ref))
    for ref in raw
]

In [11]:
with open(data_instances, 'r') as f:
    raw = json.load(f)

In [12]:
instances: Instances = Instances(**raw)

In [13]:
id2annotation: Mapping[int, Annotation] = {
    x.id: x
    for x in instances.annotations
}

Define custom dataset

In [14]:
class CocoDataset(Dataset[tuple[I, P, B]]):

    def __init__(
        self,
        split: Split,
        img_transform: Callable[[Img], I] = lambda x: x,
        prompt_transform: Callable[[list[Sentence]], P] = lambda ps: [ p.sent for p in ps ],
        bb_transform: Callable[[Float[torch.Tensor, '4']], B] = lambda x: x
    ):
        """
        :param split: train, test or val
        :param img_transform: apply transformation on the processed images
        :param prompt_transform: apply transformation on the prompts
        :param bb_transform: apply transformation on the bounding box
        """
        self.img_transform = img_transform
        self.prompt_transform = prompt_transform
        self.bb_transform = bb_transform

        # Internally the dataset is a list of tuple[str, list[Sentence], UInt[torch.Tensor, '4']]
        # Such that:
        # str                     : image filename
        # list[Sentence]          : list of reference expression objects
        # UInt[torch.Tensor, '4'] : bounding box
        self.items: list[tuple[str, list[Sentence], Float[torch.Tensor, '4']]] = [
            (i, ps, o)
            for ref in refs
            if ref.split == split
            for i in [os.path.join(data_images, ref.file_name)]
            for ps in [ref.sentences]
            for o in [torch.tensor(id2annotation[ref.ann_id].bbox, dtype=torch.float)]
        ]


    def __len__(self) -> int:
        return len(self.items)


    def __getitem__(self, item: int) -> tuple[I, P, B]:
        i, ps, b = self.items[item]
        return (
            self.img_transform(read_image(i)),
            self.prompt_transform(ps),
            self.bb_transform(b),
        )

## Training free CLIP results
For the sake of comparison with the implementations below, we have to evaluate CLIP training free with the same portion of the dataset.

Load yolo model

In [ ]:
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
yolo_model.to(device=device).eval()

Load CLIP model

In [16]:
clip_model, preprocess = clip.load('RN50')
clip_model = clip_model.to(device=device).eval()

100%|████████████████████████████████████████| 244M/244M [00:02<00:00, 111MiB/s]


Baseline evaluation

In [17]:
test_dataset: Dataset[tuple[Img, list[str], UInt[torch.Tensor, '4']]] = CocoDataset(split='test')

In [18]:
BATCH_SIZE = 1
NUM_WORKERS = os.cpu_count()
print(f"Creating DataLoader's with batch size {BATCH_SIZE} and {NUM_WORKERS} workers.")

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=False
)

Creating DataLoader's with batch size 1 and 2 workers.


In [19]:
MAX_ITER: int = 10 # max number of iterations
current_iteration: int = 0

stored_predictions: list[Prediction] = []

ious: list[float] = []
coss: list[float] = []
euds: list[float] = []

batch: tuple[UInt[torch.Tensor, '1 C W H'], tuple[list[tuple[str]]], UInt[torch.Tensor, '1 4']]

with torch.no_grad():
    for batch in tqdm(it.islice(iter(test_dataloader), min(MAX_ITER, len(test_dataloader))), total=min(MAX_ITER, len(test_dataloader))):
        [img], (prompts), [true_xywh] = batch

        [true_xyxy] = torchvision.ops.box_convert(true_xywh.unsqueeze(0), in_fmt='xywh', out_fmt='xyxy')

        img_pil: Image = transforms.ToPILImage()(img)

        # yolo bboxes
        predictions = yolo_model(img_pil)

        # xmin,      ymin,      xmax,      ymax,      confidence, class
        # 274.06390, 231.20389, 392.66345, 372.59018, 0.93251,    23.00000
        bboxes: Float[torch.Tensor, 'X 6'] = predictions.xyxy[0]

        # if empty, put a bbox equal to image size
        if len(bboxes) == 0:
            bboxes = torch.tensor([[0, 0, img.size()[1], img.size()[2], 0, 0]], dtype=torch.float)

        # from yolo bboxes to cropped images
        crops: list[Image] = [
            img_pil.crop((xmin, ymin, xmax, ymax))
            for bbox in bboxes
            for [xmin, ymin, xmax, ymax, _, _] in [bbox.tolist()]
        ]

        # clip preprocess on cropped images
        preprocess_crops: Float[torch.Tensor, 'X 3 244 244'] = torch.stack([
            preprocess(crop)
            for crop in crops
        ]).to(device=device)

        # format each available prompt
        prompts_tokens: Int[torch.Tensor, 'P 77'] = clip.tokenize([
            template.format(prompt)
            for template in ["{}", "A photo of {}", "We can see {}"]
            for (prompt,) in prompts  # <- ¯\_(ツ)_/¯
        ])

        # clip scores
        ass_z: tuple[Float[torch.Tensor, 'X P'], Float[torch.Tensor, 'P X']] = clip_model(preprocess_crops, prompts_tokens)
        _, logits_per_prompt = ass_z

        # final prediction
        best_match: int = torch.argmax(torch.max(logits_per_prompt, 0).values).item()
        prediction_bbox: Float[torch.Tensor, '4'] = bboxes[best_match][:4]

        # metrics
        iou: float = torchvision.ops.box_iou(true_xyxy.unsqueeze(0), prediction_bbox.unsqueeze(0)).item()
        ious.append(iou)

        rectangle: tuple[int, int, int, int] = true_xyxy.tolist()
        ground_truth_crop = img_pil.crop(rectangle)

        rectangle: tuple[int, int, int, int] = torch.tensor(prediction_bbox, dtype=torch.int).tolist()
        prediction_crop = img_pil.crop(rectangle)

        # from float16 to float32
        X: Float[torch.Tensor, '1'] = torch.tensor(
            clip_model.encode_image(torch.tensor(preprocess(ground_truth_crop)).unsqueeze(0)),
            dtype=torch.float
        )
        Y: Float[torch.Tensor, '1'] = torch.tensor(
            clip_model.encode_image(torch.tensor(preprocess(prediction_crop)).unsqueeze(0)),
            dtype=torch.float
        )

        cos: float = F.cosine_similarity(X, Y).item()
        coss.append(cos)

        eud: float = torch.cdist(X, Y, p=2).item()
        euds.append(eud)

        # store the prediction
        pred : Prediction = Prediction(
              image = img,
              description = [p[0] for p in prompts],
              ground_truth_bbox = true_xyxy,
              output_bbox = prediction_bbox
            )
        stored_predictions.append(pred)

        torch.cuda.empty_cache()

100%|██████████| 10/10 [00:41<00:00,  4.13s/it]


In [20]:
prompts_tokens.shape

torch.Size([6, 77])

In [21]:
preprocess_crops.shape

torch.Size([1, 3, 224, 224])

In [22]:
len(stored_predictions)

10

Performance:

In [23]:
print(f"ious: {torch.mean(torch.tensor(ious, dtype=torch.float))}")
print(f"coss: {torch.mean(torch.tensor(coss, dtype=torch.float))}")
print(f"euds: {torch.mean(torch.tensor(euds, dtype=torch.float))}")

ious: 0.8920486569404602
coss: 0.9736908078193665
euds: 0.46590811014175415


Function to display a random sample of predictions.

In [24]:
# args:
#  - predictionList: [Prediction]
#  - numPred: int :: if numPred==-1 (default) consider all the predictions in predictionList
def display_predictions(predictionList, numPred=-1):
  limit = 0
  for p in predictionList:
    if numPred!=-1 and limit >= numPred:
      return;
    limit += 1

    p_image = p.image
    p_description = p.description
    p_ground_truth_bbox = p.ground_truth_bbox
    p_output_bbox = p.output_bbox

    # TODO: concatenate
    p_image = draw_bounding_boxes(p_image, p_ground_truth_bbox.unsqueeze(0), colors="green", width=5)
    p_image = draw_bounding_boxes(p_image, p_output_bbox.unsqueeze(0), colors="red", width=5)

    tensor_to_pil = transforms.ToPILImage()
    image_pil = tensor_to_pil(p_image)
    display(image_pil)
    print(p_description)
    print("\n\n")

In [ ]:
display_predictions(stored_predictions, 3)

## Region proposal networks

### YOLOv5

In [45]:
class Yolo_v5(torch.nn.Module):
  def __init__(self, device=device):
    super().__init__()

    # load yolo model
    yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    yolo_model.to(device=device).eval()

  def forward(self, img: torch.Tensor) -> torch.Tensor:
    # convert image tensor to image PIL
    pil_transormation = transforms.ToPILImage()
    img_pil = [pil_transormation(_img) for _img in img]

    #######print(f"img_pil 0: {type(img_pil)}")
    #######print(f"img_pil 1: {len(img_pil)}")

    # yolo bboxes
    predictions = yolo_model(img_pil)
    #######print(f"predictions type: {type(predictions)}")
    #######predictions.show()

    #######print("predictions.xyxy")
    #######print(predictions.xyxy)
    #######print("predictions.xyxy len")
    #######print(len(predictions.xyxy))

    # xmin,      ymin,      xmax,      ymax,      confidence, class
    # 274.06390, 231.20389, 392.66345, 372.59018, 0.93251,    23.00000
    bboxes: list[Float[torch.Tensor, 'X 6']] = predictions.xyxy # bboxes[i] contains the bboxes highlighted by yolo in image i

    #######print("len bboxes")
    #######print(len(bboxes))

    for image_idx, bbox_img in enumerate(bboxes):
      # if empty, put a bbox equal to image size
      if len(bbox_img) == 0:
          bbox_img = torch.tensor([[0, 0, img_pil[image_idx].size()[1], img_pil[image_idx].size()[2], 0, 0]], dtype=torch.float)  # TODO: test this piece of code

    return bboxes

## Linear layers on top of image encoder

In [27]:
#summary(clip_model.visual, input_size=(1, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"])

In [28]:
#summary(clip_model.transformer, input_size=(77,512), col_names=["input_size", "output_size", "num_params", "trainable"])

In the following cell we create a neural network that builds a linear head on top of the visual encoder of CLIP.

In [29]:
class CLIP_SF_image_encoder(torch.nn.Module):
  def __init__(self):
    super().__init__()
    model, _ = clip.load("RN50")

    # take the visual encoder of CLIP
    # we also convert it to be 32 bit (by default CLIP is 16)
    self.encoder = model.visual

    # freeze all pretrained layers by setting requires_grad=False
    for param in self.encoder.parameters():
      param.requires_grad = False

    # add a linear layer
    self.fc1 = nn.Linear(1024, 1024)
    self.fc2 = nn.Linear(1024, 1024)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    # visual encoder
    with torch.no_grad():
      x = self.encoder(x)
    # ---

    # linear head
    x = self.fc1(x)
    x = F.relu(x)

    x = self.fc2(x)
    # ---

    return x

In [ ]:
net = CLIP_SF_image_encoder().to(device)
summary(net, input_size=(1, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"])

In [31]:
def get_optimizer(model, _lr, _wd, _momentum):
  optimizer = torch.optim.SGD(  params = model.parameters(),
                                lr = _lr,
                                weight_decay = _wd,
                                momentum = _momentum)

  return optimizer

In [32]:
def get_cost_function():
  def iou_loss(bbox_prediction, bbox_groundtruth):
    return -1*torchvision.ops.box_iou(bbox_groundtruth, bbox_prediction).item()

  return iou_loss

In [33]:
def get_accuracy_function():
  def iou_accuracy(bbox_prediction, bbox_groundtruth):
    return torchvision.ops.box_iou(bbox_groundtruth, bbox_prediction).item()
  return iou_accuracy

In [34]:
# deal with different size tensors in dataloader
def custom_collate(batch) -> tuple[list[Img], list[str], list[Float[torch.Tensor, '4']]]:
    images = [item[0] for item in batch]
    prompts = [item[1] for item in batch]
    bboxes = [item[2] for item in batch]

    return images, prompts, bboxes

In [71]:
def training_step(
    model: torch.nn.Module,                     # neural network to be trained
    region_proposal_model: torch.nn.Module,     # region proposal model
    data_loader: torch.utils.data.DataLoader,   # [train_dataset]
    loss_fn: torch.nn.Module,                   # todo: in our case it is not correct nn.Module, test data type
    optimizer: torch.optim.Optimizer,           # optimizer
    accuracy_fn,                                # accuracy function
    max_sample: int = -1,                       # useful during the experiments to set an upper bound on the number of samples to be evaluated (-1 :: no limit)
    device: torch.device = device               # target device
    ):

  print("training_step")
  train_loss = 0.0
  train_acc = 0.0 #todo riflettere anche su questo

  model.to(device)
  model.train()

  num_iteration: int = 0  # keep track of the number of iterations
  for batch_idx, (img, prompts, true_xywh) in tqdm(it.islice(enumerate(data_loader), (len(data_loader) if max_sample == -1 else max_sample))):

    """
    print()
    print(f"batch_idx: {batch_idx}")
    print(f"img: {img}")
    print(f"type img: {type(img)}")
    print(f"prompts: {prompts}")
    print(f"true_xywh: {true_xywh}")
    """

    # check number of iteration
    if max_sample!=-1 and num_iteration>=max_sample:
      print("\n STOP TRAINING LOOP FOR MAX ITERATION PARAMETER \n")
      break
    num_iteration+=1

    # send data to target device
    for _img in img:
      _img = _img.to(device)

    true_xywh = torch.stack((true_xywh)).to(device)

    print(f"true_xywh new: {true_xywh}")
    print(f"true_xywh new shape: {true_xywh.shape}")

    # convert bbox to the proper format
    [true_xyxy] = torchvision.ops.box_convert(true_xywh.unsqueeze(0), in_fmt='xywh', out_fmt='xyxy')

    print("len: ")
    print(len(true_xyxy))
    print(true_xyxy)

    # forward pass

    with torch.no_grad():
      # i. region proposal
      bboxes = region_proposal_model(img)

      print(f"\n\n REGION PROPOSAL ALGORITHM DONE bounding boxes: {bboxes}\n\n")

      """
      # from yolo bboxes to cropped images
      crops: list[Image] = [
          img_pil.crop((xmin, ymin, xmax, ymax))
          for bbox in bboxes
          for [xmin, ymin, xmax, ymax, _, _] in [bbox.tolist()]
      ]
      """

      # from yolo bboxes to cropped images
      crops = []
      for batch_image, batch_image_bboxes in zip(img, bboxes):
        print(f"batch_image: {batch_image.shape}")
        print(f"batch_image_bboxes: {batch_image_bboxes}")
        batch_image_pil: Image = transforms.ToPILImage()(batch_image)

        list_bboxes_image: list[Image] = [
            batch_image_pil.crop((xmin, ymin, xmax, ymax))
            for bbox in batch_image_bboxes
            for [xmin, ymin, xmax, ymax, _, _] in [bbox.tolist()]
        ]

        crops.append(list_bboxes_image)

      # ii. CLIP preprocessing  # todo: riflettere se mettere il preprocessing di CLIP nel nn.Module
      # clip preprocess on cropped images
      preproceessed_crops_batch = list()
      for image_crops in crops:
        preprocess_crops: Float[torch.Tensor, 'X 3 244 244'] = torch.stack([
            preprocess(crop)
            for crop in image_crops
        ]).to(device=device)
        preproceessed_crops_batch.append(preprocess_crops)

      """
      preprocessed_crops_tensor = torch.stack(preproceessed_crops_batch)
      print("\npreproceess_crops_batch")
      print(len(preproceessed_crops_batch))
      print(preproceessed_crops_batch[0].shape)
      print(preproceessed_crops_batch[1].shape)
      print()
      print("\preprocessed_crops_tensor")
      print(len(preprocessed_crops_tensor))
      print(preprocessed_crops_tensor.shape)
      """

      # format each available prompt
      prompts_tokens = list()
      for batch_item_prompt in prompts:

        print("!!!!batch_item_prompt!!!")
        print(batch_item_prompt)

        batch_item_prompt_tokens: Int[torch.Tensor, 'P 77'] = clip.tokenize([
            template.format(prompt)
            for template in ["{}", "A photo of {}", "We can see {}"]
            for prompt in batch_item_prompt  # <- ¯\_(ツ)_/¯
        ])
        prompts_tokens.append(batch_item_prompt_tokens)

      print("\prompts_tokens")
      print(len(prompts_tokens))
      print(prompts_tokens[0].shape)
      print(prompts_tokens[1].shape)

    # iii. process visual prompts
    model_encoding = model(preproceess_crops_batch)

    print("model_encoding")
    print(model_encoding)

    # iv. evaluate similarity bbox - prompt

    # v. final prediction

    # loss and accuracy computation

    # optimizer zero grad

    # loss backward

    # optimizer step

    ass_z: tuple[Float[torch.Tensor, 'X P'], Float[torch.Tensor, 'P X']] = clip_model(preprocess_crops, prompts_tokens)
    _, logits_per_prompt = ass_z

    # final prediction
    best_match: int = torch.argmax(torch.max(logits_per_prompt, 0).values).item()
    prediction_bbox: Float[torch.Tensor, '4'] = bboxes[best_match][:4]

    # metrics
    iou: float = torchvision.ops.box_iou(true_xyxy.unsqueeze(0), prediction_bbox.unsqueeze(0)).item()
    ious.append(iou)

    rectangle: tuple[int, int, int, int] = true_xyxy.tolist()
    ground_truth_crop = img_pil.crop(rectangle)

    rectangle: tuple[int, int, int, int] = torch.tensor(prediction_bbox, dtype=torch.int).tolist()
    prediction_crop = img_pil.crop(rectangle)

    # from float16 to float32
    X: Float[torch.Tensor, '1'] = torch.tensor(
        clip_model.encode_image(torch.tensor(preprocess(ground_truth_crop)).unsqueeze(0)),
        dtype=torch.float
    )
    Y: Float[torch.Tensor, '1'] = torch.tensor(
        clip_model.encode_image(torch.tensor(preprocess(prediction_crop)).unsqueeze(0)),
        dtype=torch.float
    )

    cos: float = F.cosine_similarity(X, Y).item()
    coss.append(cos)

    eud: float = torch.cdist(X, Y, p=2).item()
    euds.append(eud)

  # Calculate loss (todo: and accuracy) per epoch and print out what's happening
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  print(f"Train loss: {train_loss:.5f} | Train accuracy: --")
  return test_loss, test_acc

In [36]:
def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
  test_loss, test_acc = 0, 0
  model.to(device)
  model.eval() # put model in eval mode

  # Turn on inference context manager
  with torch.inference_mode():
      for X, y in data_loader:
          # Send data to GPU
          X, y = X.to(device), y.to(device)

          # 1. Forward pass
          test_pred = model(X)

          # 2. Calculate loss and accuracy
          test_loss += loss_fn(test_pred, y)
          test_acc += accuracy_fn(y_true=y,
              y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
          )

      # Adjust metrics and print out
      test_loss /= len(data_loader)
      test_acc /= len(data_loader)
      print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")
      return test_loss, test_acc

Put all together in a training loop.

In [37]:
# tensorboard logging utilities
def log_values(writer, step, loss, accuracy, prefix):
  writer.add_scalar(f"{prefix}/loss", loss, step)
  writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

In [ ]:
# instantiate the network and move it to the chosen device (GPU)
net = CLIP_SF_image_encoder().to(device)

# instantiate the region proposal algorithm
yolo = Yolo_v5().to(device)

In [72]:
# setting a manual seed allow us to provide reprudicible results in this notebook
torch.manual_seed(42)

# measure time
train_time_start = timer()  # todo: forse misurando il tempo possiamo far apprezzare la differenza di tempo di esecuzione del training quando abbiamo fatto il preprocessing delle bounding box vs senza

# create a logger for the experiment
writer = SummaryWriter(log_dir="runs/exp1")

# get dataset instance
train_dataset: Dataset[tuple[Img, list[str], UInt[torch.Tensor, '4']]] = CocoDataset(split='train')
test_dataset: Dataset[tuple[Img, list[str], UInt[torch.Tensor, '4']]] = CocoDataset(split='test')
val_dataset: Dataset[tuple[Img, list[str], UInt[torch.Tensor, '4']]] = CocoDataset(split='val')
print(f"LEN_TRAIN_DATASET: {len(train_dataset)}, LEN_TEST_DATASET: {len(test_dataset)}, LEN_VALIDATION_DATASET: {len(val_dataset)}")

# get dataloaders
BATCH_SIZE = 2
#NUM_WORKERS = os.cpu_count()
NUM_WORKERS = 1
print(f"Creating DataLoader's with batch size {BATCH_SIZE} and {NUM_WORKERS} workers.")
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    collate_fn=custom_collate,
    shuffle=True
)
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=False
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=False
)
print(f"LEN_TRAIN_DATALOADER: {len(train_loader)}, LEN_TEST_DATALOADER: {len(val_loader)}, LEN_VALIDATION_DATALOADER: {len(test_loader)}")

# instantiate the optimizer
learning_rate = 0.01
weight_decay = 0.000001
momentum = 0.9
optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)

# define the cost function
cost_function = get_cost_function()

# define the accuracy function
accuracy_fn = get_accuracy_function()

# computes evaluation results before training
print('Before training:')
#train_loss, train_accuracy = test_step(net, train_loader, cost_function)
#val_loss, val_accuracy = test_step(net, val_loader, cost_function)
#test_loss, test_accuracy = test_step(net, test_loader, cost_function)

# log to TensorBoard
#log_values(writer, -1, train_loss, train_accuracy, "train")
#log_values(writer, -1, val_loss, val_accuracy, "validation")
#log_values(writer, -1, test_loss, test_accuracy, "test")

#print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
#print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
#print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
print('-----------------------------------------------------')

epochs = 3
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_loss, train_accuracy = training_step(
        model = net,
        region_proposal_model = yolo,
        data_loader = train_loader,
        loss_fn = cost_function,
        optimizer = optimizer,
        accuracy_fn = accuracy_fn,
        max_sample = 5
    )
    val_loss, val_accuracy = test_step(data_loader=test_dataloader,
        model=model_1,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )

    # logs to TensorBoard
    log_values(writer, e, val_loss, val_accuracy, "Validation")

    print('Epoch: {:d}'.format(e+1))
    print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

train_time_end_on_gpu = timer()
total_train_time_model_1 = print_train_time(start=train_time_start_on_gpu,
                                            end=train_time_end_on_gpu,
                                            device=device)
# compute final evaluation results
print('After training:')
train_loss, train_accuracy = test_step(net, train_loader, cost_function)
val_loss, val_accuracy = test_step(net, val_loader, cost_function)
test_loss, test_accuracy = test_step(net, test_loader, cost_function)

# log to TensorBoard
log_values(writer, epochs, train_loss, train_accuracy, "train")
log_values(writer, epochs, val_loss, val_accuracy, "validation")
log_values(writer, epochs, test_loss, test_accuracy, "test")

print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
print('-----------------------------------------------------')

# closes the logger
writer.close()

LEN_TRAIN_DATASET: 42226, LEN_TEST_DATASET: 5023, LEN_VALIDATION_DATASET: 2573
Creating DataLoader's with batch size 2 and 1 workers.
LEN_TRAIN_DATALOADER: 21113, LEN_TEST_DATALOADER: 1287, LEN_VALIDATION_DATALOADER: 2512
Before training:
-----------------------------------------------------


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
---------
training_step



0it [00:00, ?it/s]

true_xywh new: tensor([[231.17999, 133.42999, 135.05000, 181.49001],
        [  2.09000, 143.55000, 198.84000, 225.61000]])
true_xywh new shape: torch.Size([2, 4])
len: 
2
tensor([[231.17999, 133.42999, 366.22998, 314.91998],
        [  2.09000, 143.55000, 200.92999, 369.16000]])


0it [00:01, ?it/s]
  0%|          | 0/3 [00:01<?, ?it/s]



 REGION PROPOSAL ALGORITHM DONE bounding boxes: [tensor([[232.87865, 133.94928, 368.31976, 317.77301,   0.90299,  23.00000],
        [452.63312,  88.27678, 506.02020, 302.33929,   0.80900,  23.00000]]), tensor([[2.79421e+02, 2.73158e+02, 4.64872e+02, 3.56336e+02, 9.51477e-01, 6.60000e+01],
        [4.55808e+02, 3.38968e+02, 4.90929e+02, 3.71173e+02, 9.21552e-01, 6.40000e+01],
        [2.40336e+02, 1.28781e+02, 4.22512e+02, 2.50913e+02, 9.04321e-01, 6.20000e+01],
        [3.22876e-01, 5.32130e+00, 1.81828e+02, 1.78073e+02, 9.01048e-01, 6.20000e+01],
        [4.27706e+02, 1.75113e+02, 6.31619e+02, 3.62734e+02, 8.22683e-01, 6.30000e+01],
        [9.42519e+00, 1.46898e+02, 1.97371e+02, 3.61229e+02, 8.12262e-01, 6.20000e+01],
        [4.22054e+02, 2.05124e+02, 4.49704e+02, 2.76256e+02, 6.74767e-01, 3.90000e+01],
        [4.65951e+02, 2.69281e+02, 5.93840e+02, 3.32168e+02, 5.00137e-01, 6.60000e+01],
        [5.47626e+02, 3.32291e+02, 6.40000e+02, 4.17268e+02, 4.13105e-01, 7.30000e+01],
   

RuntimeError: ignored

Let's evaulate our trained model using `eval_model()` function.

In [ ]:
torch.manual_seed(42)
def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn,
               device: torch.device = device):
    """Evaluates a given model on a given dataset.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.
        device (str, optional): Target device to compute on. Defaults to device.

    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            # Send data to the target device
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))

        # Scale loss and acc
        loss /= len(data_loader)
        acc /= len(data_loader)
    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}

In [ ]:
torch.manual_seed(42)

# Note: This will error due to `eval_model()` not using device agnostic code
model_1_results = eval_model(model=model_1,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn)
model_1_results

## Linear layers on top of text encoder

## Linear layers on top of image encoder and on top of text encoder

## Bottleneck